In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import caffe

caffe.set_mode_gpu()
caffe.set_device(0)

In [3]:
net = caffe.Net('../prototxt/conv.prototxt', caffe.TEST)

In [22]:
[(k, v.data.shape) for k, v in net.blobs.items()]

[('data', (1, 1, 100, 100)), ('conv', (1, 3, 96, 96))]